In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [66.5 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Pack

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-07 22:29:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-01-07 22:29:12 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

### Determine Bias of Vine Reviews

In [14]:
# Retrieve all rows where total_votes is <= 20
filtered_vine_df = vine_df.filter(vine_df['total_votes'] >= 20)
filtered_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [15]:
# Retrieve all the rows where helpful_votes divided by total_votes is >= 50%
filtered2_vine_df = filtered_vine_df.filter(filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes'] >= .5)
filtered2_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
# Retrieve all rows where vine = Y
Y_vine_df = filtered2_vine_df.filter(filtered2_vine_df['vine'] == 'Y')
Y_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [17]:
# Retrieve all rows where vine = N
N_vine_df = filtered2_vine_df.filter(filtered2_vine_df['vine'] == 'N')
N_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Calculations

In [45]:
# Calculate the total number of paid reviews
total_paid_reviews = Y_vine_df.count()
total_paid_reviews

607

In [48]:
# Calculate the total number of paid 5-star reviews
paid_5star_reviews = Y_vine_df.filter(Y_vine_df['star_rating'] == 5).count()
paid_5star_reviews

257

In [58]:
# Calculate the percent of paid 5-star reviews
percent_paid_5star_reviews = format(paid_5star_reviews / total_paid_reviews, ".1%")
percent_paid_5star_reviews

'42.3%'

In [46]:
# Calculate the total number of unpaid reviews
total_unpaid_reviews = N_vine_df.count()
total_unpaid_reviews

50522

In [23]:
# Calculate the total number of unpaid 5-star reviews
unpaid_5star_reviews = N_vine_df.filter(N_vine_df['star_rating'] == 5).count()
unpaid_5star_reviews

25220

In [57]:
# Calculate the percent of unpaid 5-star reviews
percent_unpaid_5star_reviews = format(unpaid_5star_reviews / total_unpaid_reviews, ".1%")
percent_unpaid_5star_reviews

'49.9%'

### Create a DataFrame of the results

In [25]:
# Import dependencies
import pandas as pd

In [59]:
# Create pandas dataframe
results_df = pd.DataFrame({
    'Review Type' : ['Vine', 'Non-Vine'],
    'Total Reviews' : [total_paid_reviews, total_unpaid_reviews],
    '5-Star Reviews' : [paid_5star_reviews, unpaid_5star_reviews],
    'Percent of 5-Star Reviews' : [percent_paid_5star_reviews, percent_unpaid_5star_reviews],    
})
results_df.head()

,Review Type,Total Reviews,5-Star Reviews,Percent of 5-Star Reviews
0,Vine,607,257,42.3%
1,Non-Vine,50522,25220,49.9%
